# XGBoost - модифицированный алгоритм градиентного бустинга

В основе алгоритма XGBoost лежит метод градиентного бустинга на деревьях решений. 

Принцип градиентного бустинга — это метод машинного обучения для задач классификации и регрессии, в котором  модель предсказаний строится в форме ансамбля т.н. слабых предсказывающих моделей, обученных последовательно. Как правило в качестве слабых моделей выбирают дерево решений. Среди всех моделей самая первая (назовем ее нулевой) модель вычисляет базовое предсказание $\hat{y}_0$. Оно, как правило, далеко не точное (поэтому каждый алгоритм слабый). Обозначим ошибку предсказаний как $\Delta y = y - \hat{y}_0$ На следующей итерации вычисляется вторая модель, цель которой предсказать отклонение предыдущей модели $\Delta\hat{y}_1$ (то есть провести корректировку модели). Таким образом ошибка модели становится $\Delta y = y - \left(\hat{y}_0+\Delta\hat{y}_1\right)$, в идеале ошибка должна стать меньше. На каждой следующей итерации аналогично вычисляются отклонения предсказний для результата, полученного для всех предыдущих моделей $\Delta\hat{y}^{(t)}$, таким образом: $\Delta y = y - \left(\hat{y}_0+\sum_{i=1}^t\Delta\hat{y}^{(i)}\right)$. 
Таким образом, добавив предсказания нового слабого метода к предсказаниям уже обученного ансамбля позволяет уменьшить среднее отклонение модели. Новые оценщики добавляются в ансамбль до тех пор, пока уменьшение ошибки не остановится, либо пока не выполняется одно из правил "ранней остановки".

Допустим, что мы вычисляем ошибку предсказаний как $\Delta y = l(y,  \hat{y}^0)$ для первого дерева и $\Delta y = l(y,  \hat{y}^{(t)})$ для каждого $t$ дерева. Тогда для выборки, состоящей из $N$ точек можно записать результат функции потерь как:
$$L^{(t)} = \sum_{i=1}^N l\left(y_i, \hat{y_i}^{(t-1)}+f_t\right),$$
    где:<ul>             
        <li> $f_t = \Delta\hat{y}_i^{(t)}$ - результат оценки на текущем шаге - то есть оптимизируемый слабый алгоритм, например дерево решений, для которого мы ищим глубины или другие параметры;
        <li> $\hat{y_i}^{(t-1)}$ - результат оценки на предыдущих шагах;
        </ul>  
<!-- Допустим, что результат оценки на каждом текущем шаге можно записать как $\Delta\hat{y}_i^{(t)} = w f_t,$
где $w$ - это некоторый весовой множитель, а $f^{(t)}$ - это непосредственный результат работы слабого алгоритма.  -->
Пставленная задача может быть дополнена регуляризацией с целью понижения числа слабых алгоритмов и их параметров. Для деревьев решений, этим параметром может стать глубина.
$$L^{(t)} = \sum_{i=1}^N l\left(y_i, \hat{y_i}^{(t-1)}+ f_t\right)+\Omega(f_t),$$
    где $\Omega(f)$ регуляризация функции. 
    $$\Omega(f) = \gamma T + \frac{1}{2} \lambda \lVert f \rVert ^2,$$ 
 где $T$ - количество листов в дереве (косвено и  глубина), а $\gamma$ и $\lambda$ - гиперпараметры регуляризации; $\lVert f \rVert ^2 = \sum_{j=1}^T f_{jt}^2$. 
Таким образом будем минимизровать ошибку полученную в результате невязки текущего предсказания $l$, при требовании минимальности числа листов и минимальности предсказания значений самой ошибки на текущем шаге.

В общем случае, в алгоритме XGBoost принято получать решение записанного выражения путем его разложения в ряд Тейлора до второго члена:
$$L^{(t)} = \sum_{i=1}^N l\left(y_i,\hat{y_i}^{(t-1)} + g_i f_t + \frac{1}{2} h_i f_t^2\right) + \Omega(f_t),$$ 
где
$$g_i = \frac{\partial{l(y_i,\hat{y_i}^{t-1})}}{\partial{\hat{y_i}^{t-1}}}, \quad h_i = \frac {\partial^2 {l(y_i,\hat{y_i}^{t-1})}}{\partial^2{\hat{y_i}^{t-1}}}.$$
Так, например, 
$$l = (y-\hat y)^2, g = 2(y - \hat y); h = 2.$$
$$l = y\ln\left(p\right)+\left(1-y\right)\ln\left(1-p\right); p = \frac{1}{1+e^{-x}}; $$
$$g = \frac{\partial l}{\partial x}=\frac{\partial l}{\partial p}\frac{\partial p}{\partial x}=\frac{p-y}{\left(1-p\right)p}p(1-p) = p-y; \ 
h = \frac{\partial^2 l}{\partial x^2}=\frac{\partial p}{\partial x}=p(1-p).$$

Задача построения беггинга деревьев - минимизация функционала $L$ для каждого шага $t$. В поставленной формулировке задача может быть решена методом Ньютона. 
<!-- https://neerc.ifmo.ru/wiki/index.php?title=XGBoost -->

Запишим результат функции потерь без учета постоянных слогаемых, которые не будут влиять на результат
$$L^{(t)} \approx \sum_{i=1}^n [g_i f_t + \frac{1}{2} h_i f_t^2] + \gamma T + \frac{1}{2}\lambda \sum_{j=1}^T f_{jt}^2 $$
Тогда можно периписать результат функции потерь для каждого листа как
$$L^{(t)}= \sum^T_{j=1} \left((\sum_{i\in I_j} g_i) f_{jt} + \frac{1}{2} (\sum_{i\in I_j} h_i + \lambda) f_{jt}^2 \right) + \gamma T$$
где $I_j$ - набор семплов для каждого листа при обучении дерева.

Перепишим предыдущий результат как 
$$L^{(t)} = \sum^T_{j=1} \left(G_jf_{jt} + \frac{1}{2} (H_j+\lambda) f_{jt}^2\right) +\gamma T,$$
где
$G_j = \sum_{i\in I_j} g_i$ и $H_j = \sum_{i\in I_j} h_i$.

Оптимальное значение данного уравнения (путем приравнивания проивзодной к нулю) может быть получено как
$$f_{jt}^\ast = -\frac{G_j}{H_j+\lambda}$$
тогда
$$L^{(t)\ast} = -\frac{1}{2} \sum_{j=1}^T \frac{G_j^2}{H_j+\lambda} + \gamma T$$
Таким образом можно переформулировать результат оптимизации градиентного бусинга XGB для каждого нового дерева решений как некоторый вес $f_{jt}$ с которым мы должны взять результат оценки получившегося листа $j$ данного дерева $t$ и добавить этот вес к результатам оценок предыдущих деревьев для формирования итогового предсказания.

Заметим также, что полученный критерий может быть использован также при построении самого дерева принятия решений. То есть в ходе построения дерева можно искать то, которое бы максимально оптимизировало процедуру бустинга. 
При этом критерий максимизации расщепления можно записать как
$$G = \frac{1}{2} \left[\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda}-\frac{(G_L+G_R)^2}{H_L+H_R+\lambda}\right] - \gamma $$

Теперь попробуем реализовать данную структуру



<!-- readthedocs xbg doc, paper 2016-->

<!-- https://medium.com/analytics-vidhya/what-makes-xgboost-so-extreme-e1544a4433bb -->


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns; sns.set()

import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [10]:
from ml_numpy import *

In [11]:
df = pd.read_csv("car_cat.csv")
df = df.drop(columns = ['Unnamed: 0'])

name_colums = df.columns.values
df.tail()

df = df.astype('category')

for _, column_name in enumerate(name_colums):
    df[column_name] =  df[column_name].cat.codes
    
df.info()

X,y = df.drop(columns = ['Transmission']).values,df['Transmission'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 42)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32074 entries, 0 to 32073
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Make          32074 non-null  int8 
 1   Model         32074 non-null  int16
 2   Style         32074 non-null  int8 
 3   Fuel_type     32074 non-null  int8 
 4   Transmission  32074 non-null  int8 
dtypes: int16(1), int8(4)
memory usage: 188.1 KB


Создадим изначальную выборку ответов, которую будем считать первым предсказанием 
<!-- https://medium.com/analytics-vidhya/what-makes-xgboost-so-extreme-e1544a4433bb -->

In [12]:
y_hat = np.ones_like(y)

Теперь попробуем создать класс для расчета функции потерь и ее первой и второй производных
<!-- https://medium.com/analytics-vidhya/what-makes-xgboost-so-extreme-e1544a4433bb -->

In [13]:
class LogLossWithLogits:
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    
    def activation(self,x):
        return self.sigmoid(x)

    def loss(self,y, y_hat):
        y_pred = np.clip(y_hat, 1e-15, 1 - 1e-15)
        p = self.sigmoid(y_hat)
        return y * np.log(p) + (1 - y) * np.log(1 - p)

    def gradient(self,y, y_hat):
        p = self.sigmoid(y_hat)
        return -(y - p)

    def hessian(self,y, y_hat):
        p = self.sigmoid(y_hat)
        return p * (1 - p)

также перепишим функцию расчета информативности  *_gain* =
$G^2/(H+\lambda) $
и расщепления 
$$G = \frac{1}{2} \left[\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda}-\frac{(G_L+G_R)^2}{H_L+H_R+\lambda}\right] - \gamma $$

In [14]:
def _gain(y,y_hat,lambda_, loss_func = None):
    if loss_func is None:
        loss_func = LogLossWithLogits()
    G = np.sum(loss_func.gradient(y, y_hat))
    H = np.sum(loss_func.hessian(y, y_hat))
    return G**2/(H+lambda_)

In [15]:
_gain(y,y_hat,lambda_ = 0.7)

12186.00091936532

In [16]:
def split(X, feature, threshold):
    idx_r = np.where(X[:,feature] <  threshold)[0]
    idx_l = np.where(X[:,feature] >= threshold)[0]
    return X[idx_l,:], X[idx_r,:]

def split_xy(X,y,y_hat,feature, threshold):
    Xy = np.column_stack((X,y,y_hat))
    Xy_l, Xy_r = split(Xy, feature, threshold)
    y_l     = Xy_l[:,-2]
    y_r     = Xy_r[:,-2]
    y_hat_l = Xy_l[:,-1]
    y_hat_r = Xy_r[:,-1]    
    return Xy_l[:,:-2], y_l, y_hat_l, Xy_r[:,:-2], y_r, y_hat_r

def split_y(X,y,y_hat,feature, threshold):
    Xy = np.column_stack((X,y,y_hat))
    Xy_l, Xy_r = split(Xy, feature, threshold)
    y_l     = Xy_l[:,-2]
    y_r     = Xy_r[:,-2]
    y_hat_l = Xy_l[:,-1]
    y_hat_r = Xy_r[:,-1]    
    return y_l, y_hat_l, y_r, y_hat_r

def _info_gain(y,y_hat, y_l, y_hat_l, y_r, y_hat_r, lambda_, gamma, loss_func = None):
    l   = _gain(y,  y_hat,  lambda_,loss_func)
    l_l = _gain(y_l,y_hat_l,lambda_,loss_func)
    l_r = _gain(y_r,y_hat_r,lambda_,loss_func)
    return 0.5*(l_r+l_l - l) + gamma

Проверим результат расщипления для столбца тип топлива и выбирем порог расщипления 5

In [17]:
y_l,y_hat_l,y_r,y_hat_r = split_y(X,y,y_hat,feature=3, threshold=5)
_info_gain(y,y_hat, y_l, y_hat_l, y_r, y_hat_r, lambda_=0.7, gamma=0.3, loss_func = None)

334.9330805570988

Найдем порог для лучшего расщипления

In [18]:
np.unique(X[:,3])

array([0, 1, 2, 3, 4, 5], dtype=int16)

In [19]:
loss_func = LogLossWithLogits()

for threshold in np.unique(X[:,3]):
    y_l,y_hat_l,y_r,y_hat_r = split_y(X,y,y_hat,feature=3, threshold=threshold)
    g = _info_gain(y,y_hat, y_l, y_hat_l, y_r, y_hat_r, lambda_=0.7, gamma=0.0, loss_func = loss_func)
    print(g)

3.160369271707168
103.73885401959433
98.15778417871297
62.45943366569736
6.491911523716226
334.63308055709876


как и следовало ожидать результат остался таким же как и был в предыдущем дереве. Теперь перепишем функцию поиска наилучшего расщепления

In [20]:
def best_threshold(X,y,y_hat, feature, lambda_, gamma, loss_func):
    
    thresholds = np.unique(X[:,feature])
    gains      = np.zeros(thresholds.size)

    for i,threshold in enumerate(thresholds):                    
        y1,y_hat1, y2, y_hat2 = split_y(X,y,y_hat, feature, threshold)                    
        
        if (y1.size>0) and (y2.size>0):
            gains[i] = _info_gain(y,y_hat, 
                                  y1, y_hat1, 
                                  y2, y_hat2, 
                                  lambda_, 
                                  gamma, 
                                  loss_func)
        else:
             gains[i] = 0
    
    idx_max = np.argmax(gains)            
    return thresholds[idx_max], gains[idx_max]

def best_split(X, y, y_hat, lambda_, gamma, loss_func):
    samples, features = X.shape
    gains      = np.zeros(features)
    thresholds = np.zeros(features) 
    
    for feature in range(features):
        thresholds[feature], gains[feature] = best_threshold(X,y,y_hat, feature, lambda_, gamma, loss_func)

    best_feature = np.argmax(gains)  # best index    
    
    return best_feature,thresholds[best_feature], gains[best_feature] 

In [21]:
%%time
lambda_ = 0.7
gamma   = 0.3
loss_func = LogLossWithLogits()
best_split(X, y, y_hat, lambda_, gamma, loss_func)

Wall time: 1.01 s


(0, 35.0, 1043.239899388879)

Теперь запишем базовый класс XGB дерева

In [22]:
class BasicXGBTree(BasicTree):
    def __init__(self, 
                 min_samples_split=2,
                 min_samples_leaf = 1,
                 min_gain=1e-7,
                 max_depth=-1,
                 lambda_=0.0, 
                 gamma=0.0, 
                 loss_func = None):
        
        self.lambda_ = lambda_
        self.gamma   = gamma
        self.loss_func = loss_func
        
        if self.loss_func is None:
            self.loss_func = LogLossWithLogits()
        
        super(BasicXGBTree,self).__init__(min_samples_split,
                                          min_samples_leaf,
                                          min_gain,
                                          max_depth)
        
    #-----------------------------------------------
    # Class method 
    def fit(self, X, y, y_hat):
        self.n_samples, self.n_features = X.shape
        self.root = Node(depth = 1)
        self.root = self._build(X, y, y_hat)
        return self
    
    #-----------------------------------------------
    # Class method 
    def _build(self, X, y, y_hat, depth=1):
        max_gain = 1
        best_feature   = 0
        best_threshold = 0
        n_samples, n_features = np.shape(X)

        if n_samples >= self.min_samples_split and depth <= self.max_depth:

            best_feature, best_threshold, max_gain= self._best_split(X,y, y_hat)
            
            if max_gain > self.min_gain:

                X_l,y_l,y_hat_l,X_r,y_r,y_hat_r = self._split_xy(X,y, y_hat, best_feature, best_threshold)
                
                return Node(feature=best_feature, 
                            threshold=best_threshold, 
                            left    = self._build(X_l,y_l, y_hat_l, depth + 1), #true_branch, 
                            right   = self._build(X_r,y_r, y_hat_r, depth + 1), #false_branch
                            depth   = depth,
                            samples = y.size,
                            gain    = max_gain,
                            probability = self._probabilities(y))

        
        leaf_value = self._leaf_weight(y, y_hat) 
        p = self._probabilities(y)
        return Node(value   = leaf_value, 
                    is_leaf = True, 
                    depth   = depth,
                    samples = y.size,
                    gain    = max_gain,
                    probability = p)

    #------------------------------------------
    # Generic method
    def _best_split(self,X, y, y_hat):
        best_feature, best_threshold, max_gain = 0,0,1
        return best_feature, best_threshold, max_gain

    #------------------------------------------
    def _leaf_weight(self, y, y_hat):
        return self._to_result(y)   
    #------------------------------------------
    def _split(self, X, feature, threshold):
        idx_r = np.where(X[:,feature] <  threshold)[0]
        idx_l = np.where(X[:,feature] >= threshold)[0]
        return X[idx_l,:], X[idx_r,:]
    
    #------------------------------------------ 
    def _split_xy(self,X,y,y_hat,feature, threshold):
        Xy = np.column_stack((X,y,y_hat))
        Xy_l, Xy_r = self._split(Xy, feature, threshold)
        y_l     = Xy_l[:,-2]
        y_r     = Xy_r[:,-2]
        y_hat_l = Xy_l[:,-1]
        y_hat_r = Xy_r[:,-1]    
        return Xy_l[:,:-2], y_l, y_hat_l, Xy_r[:,:-2], y_r, y_hat_r
    #------------------------------------------
    def _split_y(self,X,y,y_hat,feature, threshold):
        Xy = np.column_stack((X,y,y_hat))
        Xy_l, Xy_r = self._split(Xy, feature, threshold)
        y_l     = Xy_l[:,-2]
        y_r     = Xy_r[:,-2]
        y_hat_l = Xy_l[:,-1]
        y_hat_r = Xy_r[:,-1]    
        return y_l, y_hat_l, y_r, y_hat_r
    
    #------------------------------------------
    def _predict(self, x, tree=None):
        if tree is None:
            tree = self.root
        
        if tree.is_leaf:
            return tree.value

        if x[tree.feature] >= tree.threshold:
            branch = tree.left
        else:
            branch = tree.right

        return self._predict(x, branch) 

проверка работоспособности дерева

In [23]:
y_hat = np.zeros_like(y)
tree = BasicXGBTree(max_depth=3)
tree.fit(X,y, y_hat)
tree.print_tree()

0:0.000?
   BRANCH = 1_LEFT (32074 samples)-> 0:0.000?
      BRANCH = 2_LEFT (32074 samples)-> 0:0.000?
            BRANCH = 3_LEFT (32074 samples)-> 0 [0.54233959 0.45766041]
            BRANCH = 3_RIGHT (32074 samples)-> 0 []
      BRANCH = 2_RIGHT (32074 samples)-> 0 []
   BRANCH = 1_RIGHT (32074 samples)-> 0 []


Построим одно тестовое XGB дерево для классификации. Его отличием от реальных деревьев будет то, что мы не будем присваивать полученным листам весовые коэффициенты. В данном дереве мы проверим работоспособность критерия расщипления, записанного выше.

In [24]:
class TestClassificationXGBTRee(BasicXGBTree):
    #------------------------------------------ 
    def _best_threshold(self,X,y, y_hat, feature):

        thresholds = np.unique(X[:,feature])
        gains      = np.zeros(thresholds.size)

        for i,threshold in enumerate(thresholds):                    
            y1,y_hat1, y2, y_hat2 = self._split_y(X,y,y_hat,feature,threshold)                    

            if (y1.size>0) and (y2.size>0):
                gains[i] = self._info_gain(y,y_hat, 
                                           y1, y_hat1, 
                                           y2, y_hat2)
            else:
                 gains[i] = 0

        idx_max = np.argmax(gains)            
        return thresholds[idx_max], gains[idx_max]

    #------------------------------------------ 
    def _best_split(self,X, y, y_hat):
        samples, features = X.shape
        gains      = np.zeros(features)
        thresholds = np.zeros(features) 

        for feature in range(features):
            thresholds[feature], gains[feature] = self._best_threshold(X,y,y_hat, feature)

        best_feature = np.argmax(gains)  # best index    

        return best_feature,thresholds[best_feature], gains[best_feature]    
    
    #------------------------------------------ 
    def _info_gain(self,y,y_hat, y_l, y_hat_l, y_r, y_hat_r):
        l = self._loss(y,y_hat)
        l_l = self._loss(y_l,y_hat_l)
        l_r = self._loss(y_r,y_hat_r)
        return 0.5*(l_r+l_l - l) + self.gamma
    
    #------------------------------------------ 
    def _loss(self,y,y_hat):
        G = np.sum(self.loss_func.gradient(y, y_hat))
        H = np.sum(self.loss_func.hessian(y, y_hat))
        return G**2/(H+self.lambda_)
    
    #------------------------------------------
    def _to_result(self, y):
        return self._to_class(y)
    
    #------------------------------------------
    def _to_class(self, y):
        y = np.asarray(y)
        if y.ndim >0:
            values,counts = np.unique(y, return_counts=True)
            i_max = np.argmax(counts)
            return values[i_max]
        else:
            return y

    #---------------------------------
    def score(self, X, y):
        yhat  = self.predict(X)
        return sum((yhat==y)*1)/y.size

In [25]:
%%time
y_hat = np.zeros_like(y_train)
tree = TestClassificationXGBTRee(min_samples_split=10,
                                 min_gain=1e-02,
                                 max_depth=5,
                                 lambda_=0.5, 
                                 gamma=0.3).fit(X_train,y_train,y_hat)

Wall time: 1.29 s


In [26]:
print(tree.score(X_test,y_test),  tree.score(X_train,y_train))

0.7209809830614153 0.7156474099149258


In [27]:
tree.print_tree()

0:35.000?
   BRANCH = 1_LEFT (22451 samples)-> 0:39.000?
      BRANCH = 2_LEFT (11208 samples)-> 2:4.000?
            BRANCH = 3_LEFT (8774 samples)-> 1:113.000?
                        BRANCH = 4_LEFT (7802 samples)-> 2:8.000?
                                                BRANCH = 5_LEFT (1024 samples)-> 1 [0.34303216 0.65696784]
                                                BRANCH = 5_RIGHT (1024 samples)-> 0 [0.65229111 0.34770889]
                        BRANCH = 4_RIGHT (7802 samples)-> 3:1.000?
                                                BRANCH = 5_LEFT (6778 samples)-> 0 [0.57292528 0.42707472]
                                                BRANCH = 5_RIGHT (6778 samples)-> 0 [0.76261504 0.23738496]
            BRANCH = 3_RIGHT (8774 samples)-> 2:3.000?
                        BRANCH = 4_LEFT (972 samples)-> 1:80.000?
                                                BRANCH = 5_LEFT (730 samples)-> 1 [0.27586207 0.72413793]
                                                

Теперь допустим, что мы будем строить ансамбль деревьев. 

Как уже отмечалось для каждого дерева в ансамбле мы будем вычислять взвешенное предсказание $f_t^\ast$.

Также отметим, что для получения итогового предсказания класса мы будем использовать функцию активации сигмоид (мы учитывали ее в функции потерь) и значение порога, по которому мы будем принимать решение о классе.

Также в дерево добавлена информация о предыдущих деревьях, что необходимо для расчета итогового предсказания

In [28]:
class ClassificationXGBTRee(TestClassificationXGBTRee):
    
    def __init__(self, 
                 min_samples_split = 2,
                 min_samples_leaf  = 1,
                 min_gain          = 1e-7,
                 max_depth         = -1,
                 lambda_           = 0.0, 
                 gamma             = 0.0, 
                 class_threshold   = 0.5,
                 loss_func         = None):
        
        super(ClassificationXGBTRee,self).__init__(min_samples_split,
                                                   min_samples_leaf,
                                                   min_gain,
                                                   max_depth,
                                                   lambda_, 
                                                   gamma, 
                                                   loss_func)
        self.class_threshold = class_threshold
    
    #------------------------------------------
    def _leaf_weight(self, y, y_hat):
        G = np.sum(self.loss_func.gradient(y, y_hat))
        H = np.sum(self.loss_func.hessian(y, y_hat))
        return -G/(H+self.lambda_)

    #------------------------------------------
    def get_weights(self, x, y_previous = None):
        y_f = np.asarray([self._predict(sample) for sample in x])
        if y_previous is None:
            return y_f
        else:
            return y_f + y_previous
        
    #------------------------------------------
    def predict(self, x, y_previous = None): 
        return self.loss_func.activation(self.get_weights(x, y_previous))>= self.class_threshold
    
    #---------------------------------
    def score(self, X, y, y_previous = None):
        yhat  = self.predict(X, y_previous)
        return sum((yhat==y)*1)/y.size

Создадим первое дерево решений

In [29]:
%%time
y_hat = np.zeros_like(y_train)

tree = ClassificationXGBTRee(min_samples_split=10,
                                 min_gain=1e-02,
                                 max_depth=5,
                                 lambda_=0.5, 
                                 gamma=0.3).fit(X_train,y_train,y_hat)

Wall time: 1.32 s


In [30]:
print(tree.score(X_test,y_test),  tree.score(X_train,y_train))

0.7209809830614153 0.7156474099149258


Теперь получим значения весовых параметров для тренировочного и тестового наборов данных

In [31]:
y_hat_train = tree.get_weights(X_train)
y_hat_test  = tree.get_weights(X_test)

И теперь создадим второе дерево

In [32]:
tree2 = ClassificationXGBTRee(min_samples_split=10,
                                 min_gain=1e-02,
                                 max_depth=5,
                                 lambda_=0.5, 
                                 gamma=0.3).fit(X_train,y_train,y_hat_train)

In [33]:
print(tree2.score(X_test,y_test,y_hat_test),  tree2.score(X_train,y_train,y_hat_train))

0.767743946794139 0.7625495523584696


Видим, что точность стала выше. Попробуем еще раз

In [34]:
y_hat_train = tree2.get_weights(X_train,y_hat_train)
y_hat_test  = tree2.get_weights(X_test,y_hat_test)

tree3 = ClassificationXGBTRee(min_samples_split=10,
                                 min_gain=1e-02,
                                 max_depth=5,
                                 lambda_=0.5, 
                                 gamma=0.3).fit(X_train,y_train,y_hat_train)

In [35]:
print(tree3.score(X_test,y_test,y_hat_test),  tree3.score(X_train,y_train,y_hat_train))

0.7743946794139042 0.7671373212774487


Создадим класс для работы с XGB

In [36]:
class XGBClassifier:
    def __init__(self,
                 max_estimators    = 1,
                 min_samples_split = 2,
                 min_samples_leaf  = 1,
                 min_gain          = 1e-7,
                 max_depth         = -1,
                 lambda_           = 0.0, 
                 gamma             = 0.0, 
                 class_threshold   = 0.5,
                 min_score         = 1e-5,
                 loss_func         = None):
        
        self.max_estimators    = max_estimators
        self.min_samples_split = min_samples_split       
        self.min_samples_leaf  = min_samples_leaf     
        self.min_gain          = min_gain      
        self.max_depth         = max_depth      
        self.lambda_           = lambda_
        self.gamma             = gamma
        self.class_threshold   = class_threshold
        
        self.loss_func = loss_func
        
        if self.loss_func is None:
            self.loss_func = LogLossWithLogits()
        
        self.min_score = min_score
        self.trees     = []
        self.n_trees   = 0
    
    #----------------------------------    
    def fit(self,X,y, X_val=None,y_val=None, verbose=True):
        y_hat = np.zeros_like(y)
        self.trees = np.array([])

        if (X_val is None and y_val is None):
            val_flag  = False
        else :
            val_flag  = True
            y_hat_val = np.zeros_like(y_val)

        
        for i in range(self.max_estimators):
            tree = ClassificationXGBTRee(min_samples_split = self.min_samples_split,
                                         min_samples_leaf  = self.min_samples_leaf,
                                         min_gain          = self.min_gain,
                                         max_depth         = self.max_depth,
                                         lambda_           = self.lambda_,
                                         gamma             = self.gamma,
                                         class_threshold   = self.class_threshold,
                                         loss_func         = self.loss_func
                                         ).fit(X,y,y_hat)
            

            y_hat = tree.get_weights(X,y_hat)
            self.trees = np.append(self.trees, tree)

            if verbose:
                print('='*10)
                if val_flag:
                    y_hat_val = tree.get_weights(X_val,y_hat_val)
                    print('i = ',i,'; train score = %.4f'%tree.score(X,y,y_hat),  'val score = %.4f'%tree.score(X_val,y_val,y_hat_val))
                else:
                    print('i = ',i,'; score = %.4f'%tree.score(X,y,y_hat))

        self.n_trees = i+1
        return self
    #--------------------
    def predict(self,X):

        y_hat = np.zeros(X.shape[0])

        for i in range(self.n_trees):
            y_hat = self.trees[i].get_weights(X,y_hat)

        return self.loss_func.activation(y_hat)>= self.class_threshold
    
    #--------------------
    def score(self, X, y):
        yhat  = self.predict(X)
        return sum((yhat==y)*1)/y.size
    
    #--------------------
    def print_scores(self, X, y):
        y_hat = np.zeros(X.shape[0])

        for i in range(self.n_trees):
            y_hat = self.trees[i].get_weights(X,y_hat)   
            print('i = ',i,'; train score = %.4f'%self.trees[i].score(X,y,y_hat))
        
        print('overall score = %.4f'%self.score(X,y))


In [37]:
%%time
xgb = XGBClassifier(max_estimators    = 12,
                    min_samples_split = 10,
                    min_gain          = 1e-02,
                    max_depth         = 5,
                    lambda_           = 0.5, 
                    gamma             = 0.0
                   ).fit(X_train,y_train)

i =  0 ; score = 0.7156
i =  1 ; score = 0.7507
i =  2 ; score = 0.7659
i =  3 ; score = 0.7664
i =  4 ; score = 0.7771
i =  5 ; score = 0.7874
i =  6 ; score = 0.7905
i =  7 ; score = 0.7985
i =  8 ; score = 0.8048
i =  9 ; score = 0.8030
i =  10 ; score = 0.8059
i =  11 ; score = 0.8042
Wall time: 25.7 s


In [38]:
print(xgb.score(X_test,y_test),  xgb.score(X_train,y_train))

0.8064013301465239 0.8097189434769052


In [39]:
xgb.print_scores(X_test,y_test)

i =  0 ; train score = 0.7210
i =  1 ; train score = 0.7571
i =  2 ; train score = 0.7735
i =  3 ; train score = 0.7762
i =  4 ; train score = 0.7851
i =  5 ; train score = 0.7908
i =  6 ; train score = 0.7923
i =  7 ; train score = 0.7963
i =  8 ; train score = 0.8020
i =  9 ; train score = 0.8012
i =  10 ; train score = 0.8030
i =  11 ; train score = 0.8018
overall score = 0.8064


### Задание
Добавьте к процессу обучения процедуру ранней остановки по условию прекращения роста (или падению) точности на валидационной выборке ($X_{val}, y_{val}$).

### Задание (Необязательное)

создайте XGB дерево для задачи регрессии $L_2$
и класс XGBRegression для решения соответствующей задачи


<!-- # https://habr.com/ru/company/vk/blog/438562/m -->

